In [15]:
import cv2
import numpy as np
import pandas as pd
import os
import collections
import subprocess
import glob
from math import floor, ceil
from sklearn.utils import shuffle

from datagen import data_generator,video_to_frames
from sklearn.utils import shuffle

import tensorflow as tf
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.layers import ConvLSTM2D, Dense, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.models import Model
pf = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(pf[0],True)

In [16]:
IMAGE = (128,128)

T = 10
def video_to_frames(path_to_vid):
   
    cap = cv2.VideoCapture(path_to_vid)
    n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    frm_list = []
    counter = 0
    valid_from = ceil(0.15*n_frames)
    valid_to = floor(0.1*n_frames)
    num_frames = n_frames-valid_from-valid_to
    frame_to_count = floor(num_frames/T)
    no_of_frames_extracted = 0
    while no_of_frames_extracted < T:
        ret, frame = cap.read()
        if ret==True:
            if counter>=valid_from and (counter-valid_from)//frame_to_count==0:
                img = cv2.resize(frame,IMAGE)
                #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                #np.expand_dims(gray,axis=1)
                frm_list.append(img)
                no_of_frames_extracted+=1
                counter+=1
            else:
                counter+=1
        else:
            break
            
    return frm_list

In [17]:
frm_list = video_to_frames('/home/tarun/Kinetic600/kinetics-downloader/dataset/train/juggling_balls/_9ZQNnjgdME.mp4')

In [18]:
T=30
IMAGE = (128,128)
def video_to_frames(path_to_vid):
   
    cap = cv2.VideoCapture(path_to_vid)
    n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    frm_list = []
    counter = 0
    no_of_frames_extracted = 0
    while no_of_frames_extracted < T:
        ret, frame = cap.read()
        if ret==True:
            if counter//T==0:
                img = cv2.resize(frame,IMAGE)
                #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                #np.expand_dims(gray,axis=1)
                frm_list.append(img)
                no_of_frames_extracted+=1
                counter+=1
            else:
                counter+=1
        else:
            break
            
    return frm_list

In [19]:
frm = np.asarray(frm_list)
print(frm.shape)

(10, 128, 128, 3)


In [20]:
def data_generator(input_dir,input_class):
    X = []
    Y = []
    no_of_videos_skipped = 0
    videos = [vid for vid in os.listdir(input_dir) if vid.endswith('.mp4')]
    print("Starting at : ",str(len(videos)))
    for vid in videos:
        frame_list = video_to_frames(os.path.join(input_dir,vid))
        # print(frame_list)
        if len(frame_list)  <  T:
            no_of_videos_skipped+=1
            print('Removed')
            videos.remove(vid)

        else:
            X.append(frame_list)
            y = [0]*2  
            y[input_class] = 1  #one hot encoading
            Y.append(y)
            
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X,Y

In [21]:
X1,Y1 = data_generator('/home/tarun/Kinetic600/kinetics-downloader/dataset/train/juggling_balls',0)
X2,Y2 = data_generator('/home/tarun/Kinetic600/kinetics-downloader/dataset/train/punching_bag',1)

X_train = np.concatenate((X1, X2))
y_train = np.concatenate((Y1, Y2))

X_train,y_train = shuffle(X_train,y_train)

Starting at :  694
Starting at :  845
Removed
Removed


In [22]:
np.save('fames',X_train)
np.save('results',y_train)
print(X_train.shape)
X_test1,y_test1 = data_generator('/home/tarun/Kinetic600/kinetics-downloader/dataset/valid/juggling_balls',0)
X_test2,y_test2 = data_generator('/home/tarun/Kinetic600/kinetics-downloader/dataset/valid/punching_bag',1)

X_test = np.concatenate((X_test1, X_test2))
y_test = np.concatenate((y_test1, y_test2))

X_test,y_test = shuffle(X_test,y_test)
print(X_test.shape)

(1535, 30, 128, 128, 3)
Starting at :  40
Starting at :  41
(81, 30, 128, 128, 3)


In [23]:
X_train = X_train.reshape(1535,30,3,128,128)
X_test = X_test.reshape(81,30,3,128,128)
print(X_test.shape)
print(y_train[2])

(81, 30, 3, 128, 128)
[0 1]


In [24]:
T = 30
IMAGE = (64,64)
i = Input(shape=(T, 3, 128, 128))

def myConv(model):
    model = ConvLSTM2D(filters=64, kernel_size=(5,5), padding='same',return_sequences = True)(model)
    model = BatchNormalization()(model)
    model = Dropout(0.2)(model)
    model = ConvLSTM2D(filters=128, kernel_size=(5,5), padding='same',return_sequences = False)(model)
    model = Dropout(0.2)(model)
    model = Flatten()(model)
    model = Dense(256, activation="relu")(model)
    model = Dropout(0.3)(model)
    model = Dense(2,activation='softmax')(model)
    
    return model
model = myConv(i)
model = Model(i,model)
opt = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer= opt,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 3, 128, 128)] 0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 30, 3, 128, 64)    1229056   
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 3, 128, 64)    256       
_________________________________________________________________
dropout (Dropout)            (None, 30, 3, 128, 64)    0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 3, 128, 128)       2458112   
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 128, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 49152)             0     

In [ ]:
r = model.fit(X_train,y_train,validation_data=(X_test, y_test),batch_size=8,epochs=30)

Epoch 1/30
192/192 [==============================] - 141s 737ms/step - loss: 0.6834 - accuracy: 0.5824 - val_loss: 0.7014 - val_accuracy: 0.5185
Epoch 2/30
192/192 [==============================] - 141s 735ms/step - loss: 0.6468 - accuracy: 0.6248 - val_loss: 0.6464 - val_accuracy: 0.6296
Epoch 3/30
192/192 [==============================] - 142s 738ms/step - loss: 0.6137 - accuracy: 0.6678 - val_loss: 0.6826 - val_accuracy: 0.6049
Epoch 4/30
192/192 [==============================] - 141s 737ms/step - loss: 0.5816 - accuracy: 0.6932 - val_loss: 0.6607 - val_accuracy: 0.6173
Epoch 5/30
192/192 [==============================] - 142s 738ms/step - loss: 0.5816 - accuracy: 0.6951 - val_loss: 0.6828 - val_accuracy: 0.6173
Epoch 6/30
192/192 [==============================] - 141s 736ms/step - loss: 0.5905 - accuracy: 0.7003 - val_loss: 0.6824 - val_accuracy: 0.6543
Epoch 7/30
192/192 [==============================] - 141s 736ms/step - loss: 0.5696 - accuracy: 0.6906 - val_loss: 0.6472 -

In [ ]:
T = 30
IMAGE = (64,64)
i = Input(shape=(T, 3, 128, 128))
x = ConvLSTM2D(filters=64, kernel_size=(5,5), padding='same',return_sequences = False)(i)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = ConvLSTM2D(filters=128, kernel_size=(5,5), padding='same',return_sequences = False)(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(2,activation='softmax')(x)
opt = tf.keras.optimizers.SGD(lr=0.001)
model = Model(i,x)
model.compile(optimizer= opt,loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

r = model.fit(X_train,y_train,validation_data=(X_test, y_test),batch_size=8,epochs=30)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 30, 3, 128, 128)] 0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 3, 128, 64)        1229056   
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 128, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 24576)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               6291712   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514 

In [25]:
from tensorflow.compiler.tf2tensorrt.wrap_py_utils import get_linked_tensorrt_version
from tensorflow.compiler.tf2tensorrt.wrap_py_utils import get_loaded_tensorrt_version

print(f"Linked TensorRT version {get_linked_tensorrt_version()}")
print(f"Loaded TensorRT version {get_loaded_tensorrt_version()}")

Linked TensorRT version (6, 0, 1)
Loaded TensorRT version (6, 0, 1)
